In [34]:
import sys
import os as os
import numpy as np
import polars as pl
import pandas as pd
import geopandas as gpd
import json
import math
import joblib
import copy
import sqlite3
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error 
from pygam import LinearGAM, s

# from shapely.geometry import Point
# from shapely.ops import unary_union
# from dataclasses import dataclass, field
# from typing_extensions import List, Dict, Tuple

if 'scicore' in os.getcwd():
    path = '/scicore/home/krysiak/hocrau00/ondemand/OptimalPV_RH'
else:
    path = os.getcwd().split('\\src')[0]
    %load_ext rpy2.ipython

os.chdir(path)
from src.calibration_class import Calibration_Settings, Calibration



The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [35]:
name_dir_export_path_PY = 'c:/Models/OptimalPV_RH/data/calibration/calib_all_CH_bfs'

In [36]:
%%R
name_dir_export_path_R <- "c:/Models/OptimalPV_RH/data/calibration/calib_all_CH_bfs"

### preprep

In [37]:
if True:
    preprep_list = [
        # Calibration_Settings(), 

        Calibration_Settings(
            name_dir_export='calib_mini_debug',
            name_preprep_subsen='bfs1201',
            bfs_numbers=[1201,],
            rerun_import_and_preprp_data_TF = True,
            export_gwr_ALL_building_gdf_TF = False
        ), 
        Calibration_Settings(
            name_dir_export='calib_mini_debug',
            name_preprep_subsen='bfs1205',
            bfs_numbers=[1205,],
            rerun_import_and_preprp_data_TF = True,
            export_gwr_ALL_building_gdf_TF = False
        ), 
        # Calibration_Settings(
        #     name_dir_export='calib_mini_debug',
        #     name_preprep_subsen='bfs96',
        #     bfs_numbers=[96,],
        #     rerun_import_and_preprp_data_TF = True,
        #     export_gwr_ALL_building_gdf_TF = False
        # ), 
        # Calibration_Settings(
        #     name_dir_export='calib_mini_debug',
        #     name_preprep_subsen='bfs1033',
        #     bfs_numbers=[1033,],
        #     rerun_import_and_preprp_data_TF = True,
        #     export_gwr_ALL_building_gdf_TF = False
        # ), 
    ]

    for i_prep, prep_sett in enumerate(preprep_list):
        print('')
    #     preprep_class = Calibration(prep_sett)
    #     preprep_class.import_and_preprep_data() if preprep_class.sett.rerun_import_and_preprp_data_TF else None

    # preprep_class = Calibration(preprep_list[0])
    # preprep_class.concatenate_prerep_data()


### concat preprep

In [38]:
calib_main_sett = Calibration_Settings(
            name_dir_export='calib_all_CH_bfs',
            scicore_concat_data_path = r'/scicore/home/krysiak/hocrau00/OptimalPV_RH/data/calibration/',
            kt_numbers=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,], 
            # name_preprep_subsen=f'bfs{bfs_number}',
            # bfs_numbers=[bfs_number,], 
            n_rows_import= None,
            rerun_import_and_preprp_data_TF = True,
            export_gwr_ALL_building_gdf_TF = False)
calib_main = Calibration(calib_main_sett)
# calib_main.concatenate_prerep_data()
# calib_main.estimdf2_regression_instsize()

  START CALIBRATION: calib_all_CH_bfs
  > name_preprep_subsen: preprep_class_default_sett
  > name_calib_subscen: calib_class_default_sett



## approach 2

### data import

In [39]:
# weed out all inf or NaN values 
df = pd.read_csv(f'{name_dir_export_path_PY}/reg2_all_CH_bfs_df_approach2.csv')
df = df.dropna(subset=['BFS_NUMMER', 'TotalPower', 'elecpri_Rp_kWh', 'pvtarif_Rp_kWh', 'north_max_flaeche', 'south_max_flaeche', 'east_max_flaeche', 'west_max_flaeche'])
df.to_csv(f'{name_dir_export_path_PY}/reg2_all_CH_bfs_df_cleaned.csv', index=False)



In [40]:
%%R
if (!requireNamespace("readr", quietly = TRUE)) install.packages("readr")
if (!requireNamespace("dplyr", quietly = TRUE)) install.packages("dplyr")
if (!requireNamespace("fixest", quietly = TRUE)) install.packages("fixest")
if (!requireNamespace("tibble", quietly = TRUE)) install.packages("tibble")
if (!requireNamespace("jsonlite", quietly = TRUE)) install.packages("jsonlite")
if (!requireNamespace("tidyr", quietly = TRUE)) install.packages("tidyr")
if (!requireNamespace("ggplot2", quietly = TRUE)) install.packages("ggplot2")
if (!requireNamespace("purr", quietly = TRUE)) install.packages("purr")
if (!requireNamespace("tidyverse", quietly = TRUE)) install.packages("tidyverse")
if (!requireNamespace("randomForest", quietly = TRUE)) install.packages("randomForest")

library(readr)
library(dplyr)
library(fixest)
library(tibble)
library(jsonlite)
library(tidyr)
library(ggplot2)
library(purrr)
library(tidyverse)
library(randomForest)


R callback write-console: <class 'UnicodeDecodeError'> 'utf-8' codec can't decode byte 0xfc in position 27: invalid start byte <traceback object at 0x00000202BDB74F80>


In addition: Warning message:


In [41]:
%%R
# train / test split-------------
# train_test_split = 0.7

# import -------------
pq_files = list.files(name_dir_export_path_R, pattern = ".parquet", full.names = TRUE)

  df <- tibble(read_csv(paste0(name_dir_export_path_R, "/reg2_all_CH_bfs_df_cleaned.csv"), locale = locale(encoding = "UTF-8")))

  df <- df %>% 
    mutate(
      GBAUJ = as_factor(GBAUJ),
      GKLAS = as_factor(GKLAS),
      GSTAT = as_factor(GSTAT),
      GWAERZH1 = as_factor(GWAERZH1),
      GENH1 = as_factor(GENH1))

  idx_heatpump <- df$GWAERZH1 %in% c('7410', '7411')
  df$GWAERZH1_str <- 'no_heatpump'
  df$GWAERZH1_str[idx_heatpump] <- 'heatpump'
  df$GWAERZH1_str <- as_factor(df$GWAERZH1_str)


  # split training & test data -------------
  split_train_test <- function(df_func, df_sub_type, split_ratio = 0.7, seed = 42) {
    set.seed(seed)
    train_indices <- sample(1:nrow(df_func), size = split_ratio * nrow(df_func))
    df_train_func <- df_func[train_indices, ]
    df_test_func <- df_func[-train_indices, ]
    
    if (df_sub_type == "train"){
      df_return = df_train_func
    }
    else if (df_sub_type == "test"){
      df_return = df_test_func
    }
    else {
      stop("df_sub_type must be either 'train' or 'test'")
    } 
    return(df_return)
  }

  df_train <- split_train_test(df, "train" )
  df_test <-  split_train_test(df, "test"  )


  # filter: only residential buildings -------------
  idx_GKLAS_res3plus <- df$GKLAS %in% c("1110", "1121", "1122")
  df__res3plus <- df %>% filter(idx_GKLAS_res3plus)
  df_train_res3plus <- split_train_test(df__res3plus, "train" )
  df_test_res3plus <-  split_train_test(df__res3plus, "test"  )


  idx_GKLAS_res1to2 <- df$GKLAS %in% c("1110", "1121")
  df__res1to2 <- df %>% filter(idx_GKLAS_res1to2)
  df_train_res1to2 <- split_train_test(df__res1to2, "train" )
  df_test_res1to2 <-  split_train_test(df__res1to2, "test"  )

  idx_kwpmax20 <- df$TotalPower < 20
  df__kwpmax20 <- df %>% filter(idx_kwpmax20 & idx_GKLAS_res1to2)
  df_train_kwpmax20 <- split_train_test(df__kwpmax20, "train" )
  df_test_kwpmax20 <-  split_train_test(df__kwpmax20, "test"  )


  # export training & test data ------------§
  write_csv(df_train, paste0(getwd(), "/data/calibration/calib_all_CH_bfs/reg2_df_train.csv"))
  write_csv(df_test, paste0(getwd(), "/data/calibration/calib_all_CH_bfs/reg2_df_test.csv"))

  write_csv(df_train_res3plus,  paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_train_res3plus.csv"))
  write_csv(df_test_res3plus,   paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_test_res3plus.csv"))
  write_csv(df_train_res1to2,   paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_train_res1to2.csv"))
  write_csv(df_test_res1to2,    paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_test_res1to2.csv" ))
  write_csv(df_train_kwpmax20,  paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_train_kwpmax20.csv"))
  write_csv(df_test_kwpmax20,   paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_test_kwpmax20.csv" ))

Rows: 94433 Columns: 20
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (20): EGID, year, BFS_NUMMER, xtf_id, n_DF_UID, GAREA, GBAUJ, GKLAS, GST...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


### OLS regression

In [42]:
%%R
# fit regression model
fe11 <- feols( TotalPower ~ elecpri_Rp_kWh + pvtarif_Rp_kWh + west_max_flaeche + south_max_flaeche + east_max_flaeche, data = df_train )
fe12 <- feols( TotalPower ~ elecpri_Rp_kWh + pvtarif_Rp_kWh + west_max_flaeche + south_max_flaeche + east_max_flaeche | year, data = df_train    )

fe13 <- feols( TotalPower ~
    elecpri_Rp_kWh + pvtarif_Rp_kWh +
    west_max_flaeche + south_max_flaeche + east_max_flaeche |
    BFS_NUMMER,
    data = df_train
    )
df_test$pred_fe13 <- predict(fe13, newdata = df_test)

fe14 <- feols( TotalPower ~
    elecpri_Rp_kWh + pvtarif_Rp_kWh +
    west_max_flaeche + south_max_flaeche + east_max_flaeche | 
    BFS_NUMMER + year,
    data = df_train
    )
df_test$pred_fe14 <- predict(fe14, newdata = df_test)

fe21 <- feols(
  TotalPower ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) +
    west_max_flaeche + I(west_max_flaeche^2) +
    south_max_flaeche + I(south_max_flaeche^2) +
    east_max_flaeche + I(east_max_flaeche^2) |
    BFS_NUMMER + year,
  data = df_train
)
df_test$pred_fe21 <- predict(fe21, newdata = df_test)

fe24a <- feols(log(TotalPower) ~ elecpri_Rp_kWh + pvtarif_Rp_kWh +west_max_flaeche + south_max_flaeche + east_max_flaeche, data = df_train)
fe24b <- feols( log(TotalPower) ~  elecpri_Rp_kWh + pvtarif_Rp_kWh + west_max_flaeche + south_max_flaeche + east_max_flaeche |  year,  data = df_train )

fe24c<- feols(
    log(TotalPower) ~ 
      elecpri_Rp_kWh + pvtarif_Rp_kWh +
      west_max_flaeche + south_max_flaeche + east_max_flaeche | 
      BFS_NUMMER, 
    data = df_train
)
df_test$pred_fe24c <- predict(fe24c, newdata = df_test)

fe24d<- feols(
    log(TotalPower) ~ 
      elecpri_Rp_kWh + pvtarif_Rp_kWh +
      west_max_flaeche + south_max_flaeche + east_max_flaeche | 
      BFS_NUMMER + year,
    data = df_train
)
df_test$pred_fe24d <- predict(fe24d, newdata = df_test)

fe25a<- feols(log(TotalPower) ~ elecpri_Rp_kWh + pvtarif_Rp_kWh +west_max_flaeche + south_max_flaeche + east_max_flaeche,data = df_train)
fe25b<- feols( log(TotalPower) ~ elecpri_Rp_kWh + pvtarif_Rp_kWh + west_max_flaeche + south_max_flaeche + east_max_flaeche |  year, data = df_train)
fe25c<- feols(
    log(TotalPower) ~
      elecpri_Rp_kWh + pvtarif_Rp_kWh +
      west_max_flaeche + south_max_flaeche + east_max_flaeche | 
      BFS_NUMMER,
    data = df_train
)
df_test$pred_fe25c <- predict(fe25c, newdata = df_test)

fe25d<- feols(
    log(TotalPower) ~
      elecpri_Rp_kWh + pvtarif_Rp_kWh +
      west_max_flaeche + south_max_flaeche + east_max_flaeche | 
      BFS_NUMMER + year,
    data = df_train
)
df_test$pred_fe25d <- predict(fe25d, newdata = df_test)

fe26a <- feols( log(TotalPower) ~  elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) + pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + west_max_flaeche + I(west_max_flaeche^2) + south_max_flaeche + I(south_max_flaeche^2) + east_max_flaeche + I(east_max_flaeche^2), data = df_train)
fe26b <- feols( log(TotalPower) ~  elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) + pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + west_max_flaeche + I(west_max_flaeche^2) + south_max_flaeche + I(south_max_flaeche^2) + east_max_flaeche + I(east_max_flaeche^2) | year, data = df_train )
fe26c <- feols(
    log(TotalPower) ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) +
    west_max_flaeche + I(west_max_flaeche^2) +
    south_max_flaeche + I(south_max_flaeche^2) +
    east_max_flaeche + I(east_max_flaeche^2) |
    BFS_NUMMER,
    data = df_train
)
df_test$pred_fe26c <- predict(fe26c, newdata = df_test)

fe26d <- feols(
    log(TotalPower) ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) +
    west_max_flaeche + I(west_max_flaeche^2) +
    south_max_flaeche + I(south_max_flaeche^2) +
    east_max_flaeche + I(east_max_flaeche^2) |
    BFS_NUMMER + year,
    data = df_train
)
df_test$pred_fe26d <- predict(fe26d, newdata = df_test)


# only residential buildings of small size
fe30 <-  feols(
  TotalPower ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + 
    GBAUJ + GKLAS + GSTAT + GWAERZH1_str + 
    west_max_flaeche + I(west_max_flaeche^2) +
    south_max_flaeche + I(south_max_flaeche^2) +
    east_max_flaeche + I(east_max_flaeche^2) |
    BFS_NUMMER + year,
  data = df_train
)
df_test$pred_fe30 <- predict(fe30, newdata = df_test)

fe31 <- feols(
  TotalPower ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + 
    GBAUJ + GKLAS + GSTAT + GWAERZH1_str + 
    west_max_flaeche + I(west_max_flaeche^2) +
    south_max_flaeche + I(south_max_flaeche^2) +
    east_max_flaeche + I(east_max_flaeche^2) |
    BFS_NUMMER + year,
  data = df_train_res3plus
)
df_test_res3plus$pred_fe31 <- predict(fe31, newdata = df_test_res3plus) 

fe32 <- feols(
  TotalPower ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + 
    GBAUJ + GKLAS + GSTAT + GWAERZH1_str + 
    west_max_flaeche + I(west_max_flaeche^2) +
    south_max_flaeche + I(south_max_flaeche^2) +
    east_max_flaeche + I(east_max_flaeche^2) |
    BFS_NUMMER + year,
  data = df_train_res1to2
)
df_test_res1to2$pred_fe32 <- predict(fe32, newdata = df_test_res1to2)

fe40 <- feols(
  TotalPower ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + 
    GBAUJ + GKLAS + GSTAT + GWAERZH1_str + 
    south_max_flaeche + I(south_max_flaeche^2) +
    FLAECHE_total + I(FLAECHE_total^2) |
    BFS_NUMMER + year,
  data = df_train_res3plus
)
df_test_res3plus$pred_fe40 <- predict(fe40, newdata = df_test_res3plus)

fe41 <- feols(
  TotalPower ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + 
    GBAUJ + GKLAS + GSTAT + GWAERZH1_str + 
    south_max_flaeche + I(south_max_flaeche^2) +
    FLAECHE_total + I(FLAECHE_total^2) |  
    BFS_NUMMER + year,
  data = df_train_res1to2
)
df_test_res1to2$pred_fe41 <- predict(fe41, newdata = df_test_res1to2)


write_csv(df_test, paste0(getwd(), "/data/calibration/calib_all_CH_bfs/reg2_df_test.csv"))
write_csv(df_test_res3plus, paste0(getwd(), "/data/calibration/calib_all_CH_bfs/reg2_df_test_res3plus.csv"))
write_csv(df_test_res1to2, paste0(getwd(), "/data/calibration/calib_all_CH_bfs/reg2_df_test_res1to2.csv"))

NOTE: 1 fixed-effect singleton was removed (1 observation).
NOTE: 1/0 fixed-effect singleton was removed (1 observation).
NOTE: 1/0 fixed-effect singleton was removed (1 observation).
NOTE: 1 fixed-effect singleton was removed (1 observation).
NOTE: 1/0 fixed-effect singleton was removed (1 observation).
NOTE: 1 fixed-effect singleton was removed (1 observation).
NOTE: 1/0 fixed-effect singleton was removed (1 observation).
NOTE: 1 fixed-effect singleton was removed (1 observation).
NOTE: 1/0 fixed-effect singleton was removed (1 observation).
NOTE: 1/0 fixed-effect singleton was removed (1 observation).
NOTE: 1/0 fixed-effect singleton was removed (1 observation).
NOTE: 1/0 fixed-effect singleton was removed (1 observation).
NOTE: 1/0 fixed-effect singleton was removed (1 observation).
NOTE: 1/0 fixed-effect singleton was removed (1 observation).


In [43]:
%%R
fe_model_list <- list(
  fe11 = fe11, fe12 = fe12, fe13 = fe13, fe14 = fe14, 
  fe21 = fe21, 
  # fe22 = fe22, fe23 = fe23,
  fe24a = fe24a, fe24b = fe24b, fe24c = fe24c, fe24d = fe24d, 
  fe25a = fe25a, fe25b = fe25b, fe25c = fe25c, fe25d = fe25d,
  fe26a = fe26a, fe26b = fe26b, fe26c = fe26c, fe26d = fe26d,
  fe30 = fe30, fe31 = fe31,
  fe40 = fe40, fe41 = fe41
)

# Export model info as a named list
model_export <- lapply(fe_model_list, function(model) {
  list(
    comment = paste(deparse(formula(model)), collapse = " "),
    coefficients = coef(model)
  )
})

# Write JSON with model names as top-level keys
write_json(
  model_export,
  path = file.path(getwd(), "data/calibration/calib_all_CH_bfs/reg2_fe_model_coef.json"),
  pretty = TRUE,
  auto_unbox = TRUE
)

### ML random forest regression - full sample

In [44]:
def run_random_forest_regression(rfr_mod_name, df_suffix, rfr_settings):

    # import
    df_train_rfr = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_train{df_suffix}.csv')
    df_test_rfr  = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_test{df_suffix}.csv')

    old_pred_cols = [col for col in df_test_rfr.columns if 'pred_' in col]
    df_test_rfr_old = df_test_rfr[old_pred_cols + ['EGID', ]].copy()


    # transformations
    # df_train_rfr = df_train_rfr.drop(columns=['EGID', 'xtf_id', 'n_DF_UID']).copy()
    cols_dtypes_tupls = {
        # 'year': 'int64',
        'BFS_NUMMER': 'category',
        'GAREA': 'float64',
        # 'GBAUJ': 'int64',   
        'GKLAS': 'category',
        # 'GSTAT': 'category',
        'GWAERZH1': 'category',
        'GENH1': 'category',
        'GWAERZH1_str': 'category',
        # 'InitialPower': 'float64',
        'TotalPower': 'float64',
        'elecpri_Rp_kWh': 'float64',
        'pvtarif_Rp_kWh': 'float64',
        'FLAECHE_total': 'float64',
        'east_max_flaeche': 'float64',
        'west_max_flaeche': 'float64',
        'north_max_flaeche': 'float64',
        'south_max_flaeche': 'float64',
    }
    df_train_rfr = df_train_rfr[[col for col in cols_dtypes_tupls.keys() if col in df_train_rfr.columns]].copy()
    df_test_rfr = df_test_rfr[[col for col in cols_dtypes_tupls.keys() if col in df_test_rfr.columns]].copy()

    df_train_rfr = df_train_rfr.dropna().copy()
    df_test_rfr = df_test_rfr.dropna().copy()


    for col, dtype in cols_dtypes_tupls.items():
        df_train_rfr[col] = df_train_rfr[col].astype(dtype)
        df_test_rfr[col]  = df_test_rfr[col].astype(dtype)

    X = df_train_rfr.drop(columns=['TotalPower', ])
    y = df_train_rfr['TotalPower']

    # encode categorical variables
    cat_cols = X.select_dtypes(include=["object", "category"]).columns
    cat_cols = X.select_dtypes(include=["object", "category"]).columns
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_Arry = encoder.fit_transform(X[cat_cols].astype(str))
    encoded_df = pd.DataFrame(encoded_Arry, columns=encoder.get_feature_names_out(cat_cols))
    X = pd.concat([X.drop(columns=cat_cols).reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)


    # rf model
    if rfr_settings['run_ML_rfr_TF']:
        # rfr_model = RandomForestRegressor(
        #     n_estimators   = rfr_settings['n_estimators'],
        #     max_depth      = rfr_settings['max_depth'],
        #     random_state   = rfr_settings['random_state'],
        #     n_jobs         = rfr_settings['n_jobs'],
        # )
        # # cross validation
        # kf = KFold(n_splits=10, shuffle=True, random_state=42)
        # cv_scores = cross_val_score(rfr_model, X, y, cv=kf, scoring='neg_mean_absolute_error')
        # rfr_model.fit(X, y)

        rfr_model = RandomForestRegressor(random_state = rfr_settings['random_state'])
        param_grid = {
            'n_estimators':      rfr_settings['n_estimators'],
            'min_samples_split': rfr_settings['min_samples_split'],
            'max_depth':         rfr_settings['max_depth'],
        }
            
        grid_search = GridSearchCV(
            rfr_model,
            param_grid,
            cv=rfr_settings['cross_validation'],
            scoring='neg_mean_absolute_error',
            n_jobs=rfr_settings['n_jobs'],
            return_train_score=True,
        )
        grid_search.fit(X, y)
        rfr_model = grid_search.best_estimator_

        # save model
        joblib.dump(rfr_model, f'{name_dir_export_path_PY}/reg2_{rfr_mod_name}_model.pkl')
        joblib.dump(encoder, f'{name_dir_export_path_PY}/reg2_{rfr_mod_name}_encoder.pkl')


    # prediction
    if rfr_settings['run_ML_rfr_TF']:
        X_test = df_test_rfr.drop(columns=['TotalPower', ])
        encoded_test_array = encoder.transform(X_test[cat_cols].astype(str))
        encoded_test_df = pd.DataFrame(encoded_test_array, columns=encoder.get_feature_names_out(cat_cols))

        X_test_final = pd.concat([X_test.drop(columns=cat_cols).reset_index(drop=True), encoded_test_df.reset_index(drop=True)], axis=1)
        X_test_final = X_test_final[X.columns]

        test_preds = rfr_model.predict(X_test_final)

        df_test_rfr[f'pred_{rfr_mod_name}'] = test_preds

    else: 
        df_test_rfr[f'pred_{rfr_mod_name}'] = np.zeros(df_test_rfr.shape[0])

    df_test_rfr.to_csv(f'{name_dir_export_path_PY}/reg2_df_test{df_suffix}_RFR.csv', index=False)

    del df_train_rfr, df_test_rfr


In [45]:
run_ML_rfr_TF = True
rfr_settings = {
    'comment_settings_RandomForestRegressor()': 
        """
        n_estimators:       Defines the number of decision trees in the Random Forest.
        random_state=0:     Ensures the randomness in model training is controlled for reproducibility.
        oob_score=True:     Enables out-of-bag scoring which evaluates the model's performance using data 
                            not seen by individual trees during training
        max_depth:          The maximum depth of the tree. If None, then nodes are expanded until all 
                            leaves are pure or until all leaves contain less than min_samples_split 
                            samples

    """,
    'run_ML_rfr_TF':        True,
    'random_state':         None,    # default: None  # | None,    
    'n_jobs':               -1,      # default: None  # | -1,      
    'cross_validation':     2, 
    'n_estimators':         [100, ]  ,    # default: 100   # | 1,       
    'min_samples_split':    [5, ]    ,    # default: 2     # | 1000,    
    'max_depth':            [20, ]   ,    # default: None  # | 3,       
}

In [46]:
run_random_forest_regression(rfr_mod_name='local_nb_rfr1', df_suffix='',             rfr_settings=rfr_settings)
# run_random_forest_regression(rfr_mod_name='local_nb_rfr2', df_suffix='_kwpmax20',    rfr_settings=rfr_settings)

### GAM - Generalized Additive Models

In [47]:
df_train = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_train.csv')
df_test = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_test.csv')

features = ['elecpri_Rp_kWh', 'pvtarif_Rp_kWh',
            'north_max_flaeche', 'south_max_flaeche',
            'east_max_flaeche', 'west_max_flaeche']

x_train = df_train[features].values
y_train = df_train['TotalPower'].values

x_test = df_test[features].values
y_test = df_test['TotalPower'].values

gam1 = LinearGAM(
    s(0) + s(1) + s(2) + s(3) + s(4) + s(5)
).fit(x_train, y_train)
y_pred_gam1 = gam1.predict(x_train)
df_test['pred_gam1'] = gam1.predict(x_test)

# export coefs and df_test
model_formula = "TotalPower ~ s(elecpri_Rp_kWh) + s(pvtarif_Rp_kWh) + s(north_max_flaeche) + s(south_max_flaeche) + s(east_max_flaeche) + s(west_max_flaeche)"
coefficients = gam1.coef_.tolist()
lambdas = list(gam1.lam)
terms = [str(gam1.terms[i]) for i in range(len(gam1.terms))]
reg2_gam_model_coef = {
    "comment": model_formula,
    "coefficients": coefficients,
    "lambdas": lambdas,
    "terms": terms
}
with open(f'{name_dir_export_path_PY}/reg2_gam_model_coef.json', 'w') as f:
    json.dump(reg2_gam_model_coef, f, indent=4)

df_test.to_csv(f'{name_dir_export_path_PY}/reg2_df_test.csv', index=False)



In [48]:
df_train_res1to2 =  pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_train_res1to2.csv')
df_test_res1to2 =   pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_test_res1to2.csv' )

features = ['elecpri_Rp_kWh', 'pvtarif_Rp_kWh',
            'north_max_flaeche', 'south_max_flaeche',
            'east_max_flaeche', 'west_max_flaeche']

x_train = df_train_res1to2[features].values
y_train = df_train_res1to2['TotalPower'].values

x_test = df_test_res1to2[features].values
y_test = df_test_res1to2['TotalPower'].values

gam2 = LinearGAM(
    s(0) + s(1) + s(2) + s(3) + s(4) + s(5)
).fit(x_train, y_train)
y_pred_gam2 = gam2.predict(x_train)
df_test_res1to2['pred_gam2'] = gam2.predict(x_test)

# export coefs and df_test
model_formula = "TotalPower ~ s(elecpri_Rp_kWh) + s(pvtarif_Rp_kWh) + s(north_max_flaeche) + s(south_max_flaeche) + s(east_max_flaeche) + s(west_max_flaeche)"
coefficients = gam2.coef_.tolist()
lambdas = list(gam2.lam)
terms = [str(gam2.terms[i]) for i in range(len(gam2.terms))]
reg2_gam_model_coef = {
    "comment": model_formula,
    "coefficients": coefficients,
    "lambdas": lambdas,
    "terms": terms
}
with open(f'{name_dir_export_path_PY}/reg2_gam_model_coef.json', 'w') as f:
    json.dump(reg2_gam_model_coef, f, indent=4)

df_test_res1to2.to_csv(f'{name_dir_export_path_PY}/reg2_df_test_res1to2.csv', index=False)


### visualization

In [49]:
df_test = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_test.csv')


In [50]:
# PREDICTION ACCURACY PLOTS
if True: 
    pred_colname_dfname_tuples = [
        ('pred_fe21', 'reg2_df_test.csv'),
        ('pred_fe31', 'reg2_df_test_res3plus.csv'),
        ('pred_fe32', 'reg2_df_test_res1to2.csv'),
        ('pred_fe40', 'reg2_df_test_res3plus.csv'),
        ('pred_fe41', 'reg2_df_test_res1to2.csv'),
        ('pred_local_nb_rfr1', 'reg2_df_test_RFR.csv'),
        ('pred_local_nb_rfr2', 'reg2_df_test_kwpmax20_RFR.csv'),
        ('pred_rfr1', 'df_test_rfr_rfr1.csv'),
        ('pred_rfr2', 'df_test_rfr_rfr2.csv'),
    ]

    # accuracy plot ------------
    pred_cols = [tup[0] for tup in pred_colname_dfname_tuples]
    ncols = 3
    nplots = len (pred_cols)
    nrows = math.ceil(nplots / ncols)
    fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=pred_cols, horizontal_spacing=0.1, vertical_spacing=0.15)

    for i, (pred_col, df_name) in enumerate(pred_colname_dfname_tuples):
        row = (i // ncols) + 1
        col = (i % ncols) + 1

        df_import = pd.read_csv(f'{name_dir_export_path_PY}/{df_name}')

        df_import = df_import.dropna(subset=[pred_col, 'TotalPower'])    
        df_accu = df_import.replace([np.inf, -np.inf], np.nan).dropna(subset=['TotalPower', pred_col]).copy()
        
        fig.add_trace(
            go.Scatter(
                x=df_accu[pred_col],
                y=df_accu['TotalPower'],
                mode='markers',
                marker=dict(size=3, opacity=0.5),
                name=pred_col,
                showlegend=False
            ),
            row=row,
            col=col
        )
        # add diagonal line y=x
        min_val = min(df_accu['TotalPower'].min(), df_accu[pred_col].min()) * 0.95
        max_val = max(df_accu['TotalPower'].max(), df_accu[pred_col].max()) * 1.05
        fig.add_trace(go.Scatter(x=[min_val, max_val],
                                    y=[min_val, max_val],
                                    mode='lines',
                                    line=dict(color='red', dash='dash'),
                                    name='Perfect Prediction',
                                    showlegend=False),
                        row=row, col=col)

        fig.update_xaxes(title_text=f'PredPwr {df_accu.shape[0]}n', row=row, col=col)
        fig.update_yaxes(title_text='Total Power', row=row, col=col)

    # layout 
    fig.update_layout(
        # height=300 * nrows, width=1000,
                    title_text=f'Predicted Power vs Total Power ({df_accu.shape[0]} n_df_train, {df_accu.shape[0]} n_df_test)',
                    plot_bgcolor='white')
    for i in range(1, nplots + 1):
        row = (i - 1) // ncols + 1
        col = (i - 1) % ncols + 1
        # fig.update_xaxes(title_text='Predicted Power', row=row, col=col)
        # fig.update_yaxes(title_text='Total Power', row=row, col=col)

    fig.write_html(f'{name_dir_export_path_PY}/reg2_pred_vs_total_power.html')


    # RMSE plot ------------
    rmse_dict = {}

    for i, (pred_col, df_name) in enumerate(pred_colname_dfname_tuples):

        df_import = pd.read_csv(f'{name_dir_export_path_PY}/{df_name}') 

        df_import = df_import.dropna(subset=[pred_col, 'TotalPower'])    
        df_accu = df_import.replace([np.inf, -np.inf], np.nan).dropna(subset=['TotalPower', pred_col]).copy()
        
        rmse = mean_squared_error(df_accu['TotalPower'], df_accu[pred_col])
        rmse_dict[pred_col] = rmse

    bar_fig = go.Figure()

    bar_fig.add_trace(
        go.Bar(
            x=list(rmse_dict.keys()),
            y=list(rmse_dict.values()),
            marker=dict(color='cornflowerblue'),
            text=[f"{v:.2f}" for v in rmse_dict.values()],
            textposition='auto'
        )
    )

    bar_fig.update_layout(
        title='RMSE of Model Predictions on Test Set',
        xaxis_title='Model',
        yaxis_title='RMSE',
        plot_bgcolor='white'
    )

    # Save bar plot
    bar_fig.write_html(f'{name_dir_export_path_PY}/reg2_rmse_barplot.html')



## explore missing pvs in reg2
find out more about (huge) amount of missing pv inst in `reg2_all_CH_bfs_df_approach2`

In [ ]:
# COMPARISON PV IN ESTIMATING SAMPLE 

# gm_shp_gdf
# gm_shp_gdf = gpd.read_file(f'{self.sett.data_path}/input/swissboundaries3d_2023-01_2056_5728.shp', layer ='swissBOUNDARIES3D_1_4_TLM_HOHEITSGEBIET')
gm_shp_gdf = gpd.read_file(R"C:\Models\OptimalPV_RH\data\input\swissboundaries3d_2023-01_2056_5728.shp\swissBOUNDARIES3D_1_4_TLM_HOHEITSGEBIET.shp", layer ='swissBOUNDARIES3D_1_4_TLM_HOHEITSGEBIET')
gm_shp_gdf = gm_shp_gdf.drop(columns = ['DATUM_AEND', 'DATUM_ERST'])
gm_shp_gdf = gm_shp_gdf.loc[gm_shp_gdf['KANTONSNUM'].notna()]
gm_shp_gdf['KANTONSNUM'] = gm_shp_gdf['KANTONSNUM'].astype(int)
gm_shp_gdf['BFS_NUMMER'] = gm_shp_gdf['BFS_NUMMER'].astype(str)
gm_shp_gdf.crs = 'EPSG:2056'

def attach_bfs_to_spatial_data(gdf, gm_shp_df, keep_cols = ['BFS_NUMMER', 'geometry' ]):
    """
    Function to attach BFS numbers to spatial data sources
    """
    gdf = copy.deepcopy(gdf)
    gdf.set_crs(gm_shp_df.crs, allow_override=True, inplace=True)
    gdf = gpd.sjoin(gdf, gm_shp_df, how="left", predicate="within")
    dele_cols = ['index_right'] + [col for col in gm_shp_df.columns if col not in keep_cols]
    gdf.drop(columns = dele_cols, inplace = True)
    if 'BFS_NUMMER' in gdf.columns:
        # transform BFS_NUMMER to str, np.nan to ''
        gdf['BFS_NUMMER'] = gdf['BFS_NUMMER'].apply(lambda x: '' if pd.isna(x) else str(int(x)))

    return gdf


# import rfr 
rfr_pl = pl.read_csv(r"C:\Models\OptimalPV_RH\data\calibration\PVALLOC_calibration_model_coefs\reg2_all_CH_bfs_df_approach2.csv")
rfr_pl = rfr_pl.with_columns([
    pl.col('xtf_id').cast(pl.Utf8),
    pl.col('BFS_NUMMER').cast(pl.Utf8),
    pl.col('GKLAS').cast(pl.Utf8),
    pl.col('GSTAT').cast(pl.Utf8),
    pl.col('GWAERZH1').cast(pl.Utf8),
])


# import pv
elec_prod_gdf = gpd.read_file(
    r"C:\Models\OptimalPV_RH\data\input\ch.bfe.elektrizitaetsproduktionsanlagen_gpkg\ch.bfe.elektrizitaetsproduktionsanlagen.gpkg",
    layer ='ElectricityProductionPlant', 
    )
pv_all_gdf = elec_prod_gdf.loc[elec_prod_gdf['SubCategory'] == 'subcat_2'].copy()
pv_all_gdf['xtf_id'] = pv_all_gdf['xtf_id'].astype(str)
pv_all_gdf = attach_bfs_to_spatial_data(pv_all_gdf, gm_shp_gdf)
pv_all_pl = pl.DataFrame(pv_all_gdf.drop(columns='geometry'))


# import gwr
query_columns = ['EGID', 'GDEKT', 'GGDENR', 'GKODE', 'GKODN', 'GKSCE', 'GSTAT', 'GKAT', 'GKLAS', 'GBAUJ', 'GBAUM', 'GBAUP', 'GABBJ', 'GANZWHG','GWAERZH1', 'GENH1', 'GWAERSCEH1', 'GWAERDATH1', 'GEBF', 'GAREA']
query_columns_str = ', '.join(query_columns)
query_bfs_numbers = ', '.join([str(i) for i in self.sett.bfs_numbers])

conn = sqlite3.connect(r"C:\Models\OptimalPV_RH\data\input/GebWohnRegister.CH/data.sqlite")
cur = conn.cursor()
cur.execute(f'SELECT {query_columns_str} FROM building')
sqlrows = cur.fetchall()
conn.close()

gwr_all_building_df = pd.DataFrame(sqlrows, columns=query_columns)

def gwr_to_gdf(df):
    df = copy.deepcopy(df)                
    df = df.loc[(df['GKODE'] != '') & (df['GKODN'] != '')]
    df[['GKODE', 'GKODN']] = df[['GKODE', 'GKODN']].astype(float)
    df['geometry'] = df.apply(lambda row: Point(row['GKODE'], row['GKODN']), axis=1)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.crs = 'EPSG:2056'
    return gdf

In [67]:
# print('pv_all_pl')
# for col in pv_all_pl.columns:
#     print(f'{col:25} type: {pv_all_pl[col].dtype}')

# print('\nrfr_pl')
# for col in rfr_pl.columns:
#     print(f'{col:25} type: {rfr_pl[col].dtype}')

pv_not_rfr = pv_all_pl.filter(~pl.col('xtf_id').is_in(rfr_pl['xtf_id'].implode()))

print(f'n_pv not in calibration: {pv_not_rfr.shape[0]/pv_all_pl.shape[0]*100:.1f}% -> {pv_not_rfr.shape[0]} not in rfr of {pv_all_pl.shape[0]} pv_all_pl installations')
bfs_list, n_pv_bfs_list, n_pv_not_rfr_bfs_list, share_pv_not_rfr_list = [], [], [], []
for bfs_num in gm_shp_gdf['BFS_NUMMER'].unique():
    n_pv_bfs = pv_all_pl.filter(pl.col('BFS_NUMMER') == bfs_num).shape[0]
    n_pv_not_rfr_bfs = pv_not_rfr.filter(pl.col('BFS_NUMMER') == bfs_num).shape[0]

    bfs_list.append(bfs_num)
    n_pv_bfs_list.append(n_pv_bfs)
    n_pv_not_rfr_bfs_list.append(n_pv_not_rfr_bfs)
    share_pv_not_rfr_list.append(n_pv_not_rfr_bfs / n_pv_bfs if n_pv_bfs > 0 else 0)

bfs_pv_nan_df = pd.DataFrame({
    'BFS_NUMMER': bfs_list,
    'n_pv_bfs': n_pv_bfs_list,
    'n_pv_not_rfr_bfs': n_pv_not_rfr_bfs_list,
    'share_pv_not_rfr': share_pv_not_rfr_list,
    })
print(f'n_bfs completely left out (n_pv in gm == n_pv not in rfr): {bfs_pv_nan_df.loc[bfs_pv_nan_df["n_pv_bfs"] == bfs_pv_nan_df["n_pv_not_rfr_bfs"]].shape[0]/bfs_pv_nan_df.shape[0]*100:.1f}% -> {bfs_pv_nan_df.loc[bfs_pv_nan_df["n_pv_bfs"] == bfs_pv_nan_df["n_pv_not_rfr_bfs"]].shape[0]} of {bfs_pv_nan_df.shape[0]} bfs areas')



n_pv not in calibration: 93.1% -> 222661 not in rfr of 239214 pv_all_pl installations
n_bfs completely left out (n_pv in gm == n_pv not in rfr): 58.1% -> 1249 of 2149 bfs areas


In [75]:
# charactersitics of pv in bfs which are not completely omitted
bfs_complete_nan = bfs_pv_nan_df.loc[bfs_pv_nan_df['share_pv_not_rfr'] == 1.0, 'BFS_NUMMER'].tolist()
pv2 = pv_all_pl.filter( 
    (~pl.col('BFS_NUMMER').is_in(bfs_complete_nan)) & 
    (~pl.col('xtf_id').is_in(rfr_pl['xtf_id'].implode()))
    )
pv2.shape
pv2.head()

# print(f'n_pv not in calibration (only bfs with at least one pv in rfr): 
# bfs_pv_nan_df = pl.DataFrame(bfs_pv_nan_dict).transpose().with_columns([
#     pl.col('n_pv_bfs').cast(pl.Int64),
#     pl.col('n_pv_not_rfr_bfs').cast(pl.Int64),
#     pl.col('share_pv_not_rfr').cast(pl.Float64),    
# ])


xtf_id,Address,PostCode,Municipality,Canton,BeginningOfOperation,InitialPower,TotalPower,MainCategory,SubCategory,PlantCategory,BFS_NUMMER
str,str,i32,str,str,str,f32,f32,str,str,str,str
"""10797""","""Brüggelbachstrasse 9""",3176,"""Neuenegg""","""BE""","""2008-11-20""",12.5,12.5,"""maincat_2""","""subcat_2""","""plantcat_9""","""670"""
"""11795""","""Via Industria 1""",6934,"""Bioggio""","""TI""","""2009-08-17""",124.199997,1541.079956,"""maincat_2""","""subcat_2""","""plantcat_8""","""5151"""
"""14763""","""Kürzematt 12""",4515,"""Oberdorf SO""","""SO""","""2009-12-07""",6.3,6.3,"""maincat_2""","""subcat_2""","""plantcat_8""",""""""
"""14764""","""Winkel""",4234,"""Zullwil""","""SO""","""2008-11-07""",3.2,3.2,"""maincat_2""","""subcat_2""","""plantcat_8""",""""""
"""14775""","""Katzacker 9""",3368,"""Bleienbach""","""BE""","""2009-11-25""",4.2,4.2,"""maincat_2""","""subcat_2""","""plantcat_8""","""324"""


In [18]:
# # SUMMARY BAR PLOTS
# df_test = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_test.csv')

# cols_to_plot_tupls = [
#     ('GAREA',              'float',   30  ), 
#     ('GBAUJ',              'str',     30  ), 
#     ('GKLAS',              'str',     30  ), 
#     ('GSTAT',              'str',     30  ), 
#     ('GENH1',              'str',     30  ), 
#     ('GWAERZH1',           'str',     30  ), 
#     ('FLAECHE_total',     'float',    30  ),
#     ('elecpri_Rp_kWh',     'float',   30  ), 
#     ('TotalPower',         'float',   30  ), 
#     ('pvtarif_Rp_kWh',     'float',   30  ), 
#     ('west_max_flaeche',   'float',   30  ), 
#     ('east_max_flaeche',   'float',   30  ), 
#     ('south_max_flaeche',  'float',   30  ), 
# ]

# ncols = 3
# nplots = len (cols_to_plot_tupls)
# nrows = math.ceil(nplots / ncols)
# subplot_titles = [col for col, _, _ in cols_to_plot_tupls]
# fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=subplot_titles, horizontal_spacing=0.1, vertical_spacing=0.15)

# for i, (col, col_type, col_nbins) in enumerate(cols_to_plot_tupls):
#     row = (i // ncols) + 1
#     col_pos = (i % ncols) + 1
#     # print(f'col: {col}, type: {col_type}, bins: {col_nbins}')

#     if col_type == 'float':
#         sub_fig = px.histogram(df_test, x=col, histnorm='probability density', nbins=col_nbins).data[0]
#         fig.add_trace(sub_fig, row=row, col=col_pos)
#     elif col_type == 'str':
#         sub_fig = px.bar(df_test, x=col).data[0]
#         fig.add_trace(sub_fig, row=row, col=col_pos)
    

# fig.update_layout(title_text="Summary Plots of Test Data", 
#                   template="plotly_white",)
# fig.write_html(f'{name_dir_export_path_PY}/reg2_df_test_summary_plots.html')
                    

In [19]:
# PARTIAL DEPENDENCIES -- FE21 OLS -- 
if False: 

    with open(f'{name_dir_export_path_PY}/reg2_fe_model_coef.json', 'r') as f:
        reg2_fe_model_coef = json.load(f)

    # call coefficients + build plot
    fe21_coef = reg2_fe_model_coef['fe21']['coefficients']
    fe21_coef_lin = [fe21_coef[0], fe21_coef[2], fe21_coef[4], fe21_coef[6], fe21_coef[8], ]
    fe21_coef_quad = [fe21_coef[1], fe21_coef[3], fe21_coef[5], fe21_coef[7], fe21_coef[9], ]

    covariates_fe21 = [
        "elecpri_Rp_kWh",
        "pvtarif_Rp_kWh",
        "west_max_flaeche",
        "south_max_flaeche",
        "east_max_flaeche",
    ]
    ncols = 3
    nplots = len (covariates_fe21)
    nrows = math.ceil(nplots / ncols)
    fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=covariates_fe21, horizontal_spacing=0.1, vertical_spacing=0.15)


    # addjust fixed effects
    agg_methods = {covar: 'mean' for covar in covariates_fe21}
    df_plot = df_test.copy()
    df_means = df_test.groupby(['BFS_NUMMER', 'year']).agg({**agg_methods, 'TotalPower': 'mean'}).reset_index()
    df_plot = df_plot.merge(df_means, on=['BFS_NUMMER', 'year'], suffixes=('', '_mean'))
    for covar in covariates_fe21:
        df_plot[f'{covar}_fe'] = df_plot[f'{covar}'] - df_plot[f'{covar}_mean']
    df_plot['TotalPower'] = df_plot['TotalPower'] - df_plot['TotalPower_mean']


    for i, covar in enumerate(covariates_fe21):
        x_array = np.linspace(0, round(max(df_test[covar]),0), 200)
        y_array = fe21_coef_lin[i] * x_array + fe21_coef_quad[i] * (x_array ** 2) 

        row = (i // ncols) + 1
        col = (i % ncols) + 1
        fig.add_trace(
            go.Scatter(
                x=x_array,
                y=y_array,
                mode='lines',
                showlegend=False
            ),
            row=row,
            col=col
        )
        fig.add_trace(
            go.Scatter(
                x=df_plot[f'{covar}'],
                y=df_plot['TotalPower'],
                mode='markers', 
                marker=dict(size=2, opacity=0.5),
                name=col,
                showlegend=False
            ),
            row=row,
            col=col
            )
        fig.update_xaxes(title_text= f'{col}', row=row, col=col)
        fig.update_yaxes(title_text='Total Power', row=row, col=col)
        
    for i in range(1, nplots + 1):
        row = (i - 1) // ncols + 1
        col = (i - 1) % ncols + 1
        # fig.update_xaxes(title_text= f'{col}', row=row, col=col)
        # fig.update_yaxes(title_text='Total Power', row=row, col=col)


    fig.update_layout(
        title_text=f'FE21 Partial Dependence Plots - {df_train.shape[0]} n_df_train, {df_test.shape[0]} n_df_test / {df_plot.shape[0]} n_df_test_fixedef',  
        plot_bgcolor='white'
    )
    fig.write_html(f'{name_dir_export_path_PY}/reg2_fe21_partial_dependence.html')

In [20]:
# PARTIAL DEPENDENCIES -- FE31 OLS -- 
if False: 
    df_train_res3plus = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_train_res3plus.csv')
    df_test_res3plus = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_test_res3plus.csv')

    with open(f'{name_dir_export_path_PY}/reg2_fe_model_coef.json', 'r') as f:
        reg2_fe_model_coef = json.load(f)

    # call coefficients + build plot
    fe31_coef = reg2_fe_model_coef['fe31']['coefficients']
    fe31_coef_lin = [fe31_coef[0], fe31_coef[2], fe31_coef[4], fe31_coef[5], fe31_coef[6], fe31_coef[7], fe31_coef[8], fe31_coef[9], 
                     fe31_coef[11], fe31_coef[13],]

    fe31_coef_quad = [ fe31_coef[1], fe31_coef[3], 0, 0, 0, 0, 0, 0, fe31_coef[10], fe31_coef[12], fe31_coef[14]]

    covariates_fe31 = [

        "elecpri_Rp_kWh",
        "pvtarif_Rp_kWh",
        "GBAUJ", 
        "GKLAS", 
        "GSTAT", 
        "GWAERZH1", 
        "GENH1", 
        "west_max_flaeche",
        "south_max_flaeche",
        "east_max_flaeche",
    ]
    ncols = 3
    nplots = len (covariates_fe31)
    nrows = math.ceil(nplots / ncols)
    fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=covariates_fe31, horizontal_spacing=0.1, vertical_spacing=0.15)


    # addjust fixed effects
    agg_methods = {covar: 'mean' for covar in covariates_fe31}
    df_plot = df_test_res3plus.copy()
    df_means = df_test_res3plus.groupby(['BFS_NUMMER', 'year']).agg({**agg_methods, 'TotalPower': 'mean'}).reset_index()
    df_plot = df_plot.merge(df_means, on=['BFS_NUMMER', 'year'], suffixes=('', '_mean'))
    for covar in covariates_fe31:
        df_plot[f'{covar}_fe'] = df_plot[f'{covar}'] - df_plot[f'{covar}_mean']
    df_plot['TotalPower'] = df_plot['TotalPower'] - df_plot['TotalPower_mean']


    for i, covar in enumerate(covariates_fe31):
        x_array = np.linspace(0, round(max(df_test_res3plus[covar]),0), 200)
        y_array = fe31_coef_lin[i] * x_array + fe31_coef_quad[i] * (x_array ** 2) 

        row = (i // ncols) + 1
        col = (i % ncols) + 1
        fig.add_trace(
            go.Scatter(
                x=x_array,
                y=y_array,
                mode='lines',
                showlegend=False
            ),
            row=row,
            col=col
        )
        fig.add_trace(
            go.Scatter(
                x=df_plot[f'{covar}'],
                y=df_plot['TotalPower'],
                mode='markers', 
                marker=dict(size=2, opacity=0.5),
                name=col,
                showlegend=False
            ),
            row=row,
            col=col
        )
    for i in range(1, nplots + 1):
        row = (i - 1) // ncols + 1
        col = (i - 1) % ncols + 1
        fig.update_xaxes(title_text= col, row=row, col=col)
        fig.update_yaxes(title_text='Total Power', row=row, col=col)


    fig.update_layout(
        title_text=f'FE21 Partial Dependence Plots - {df_train_res3plus.shape[0]} n_df_train, {df_test_res3plus.shape[0]} n_df_test / {df_plot.shape[0]} n_df_test_fixedef',  
        plot_bgcolor='white'
    )
    fig.write_html(f'{name_dir_export_path_PY}/reg2_fe31_partial_dependence.html')



In [21]:
# PARTIAL DEPENDENCIES -- GAM -- 
if False:
    df_test = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_test.csv')

    features = ['elecpri_Rp_kWh', 'pvtarif_Rp_kWh',
                'north_max_flaeche', 'south_max_flaeche',
                'east_max_flaeche', 'west_max_flaeche']

    ncols = 3
    nplots = len(features)
    nrows = math.ceil(nplots / ncols)

    fig = make_subplots(
        rows=nrows,
        cols=ncols,
        subplot_titles=features,
    )

    for i, feature in enumerate(features):
        row = (i // ncols) + 1
        col = (i % ncols) + 1

        # Generate grid of values for feature i
        XX = gam1.generate_X_grid(term=i, n=200)

        # Predict partial dependence
        pdep = gam1.partial_dependence(term=i, X=XX)

        # Scatter raw data for visual context
        fig.add_trace(
            go.Scatter(
                x=df_test[feature],
                y=df_test['TotalPower'],
                mode='markers',
                marker=dict(size=2, opacity=0.3),
                showlegend=False
            ),
            row=row,
            col=col
        )

        # Add partial dependence line
        fig.add_trace(
            go.Scatter(
                x=XX[:, i],
                y=pdep,
                mode='lines',
                line=dict(color='red'),
                name=f'PDP: {feature}',
                showlegend=False
            ),
            row=row,
            col=col
        )

        fig.update_xaxes(title_text=feature, row=row, col=col)
        fig.update_yaxes(title_text='Partial Effect', row=row, col=col)

    # Layout
    fig.update_layout(
        title_text=f'GAM Partial Dependence Plots ({df_train.shape[0]} train, {df_test.shape[0]} test)',
        plot_bgcolor='white'
    )

    # Save HTML output
    fig.write_html(f'{name_dir_export_path_PY}/reg2_gam_partial_dependence.html')